# Final Assignment Week 4 - Electric charges infrastructure in Puebla City, Mexico.

### Introduction

##### The electric car sales will be increased in the following years. As a following effect, for every place  a car can reach, the  electric chargers will be required.
##### Nowadays just few brands from electrics cars can be seen on the streets. In Mexico, electric cars market is just starting. Currently is quite strange to see those cars
##### unless you are located in wealthy neighborhoods. On the other side, there are already electric cars production projects in almost most of OEM's. So it's almost a fact, that
##### electric cars will be  mass produced. 

In [3]:
Image(url= "https://canalys-com-public-prod.s3.eu-west-2.amazonaws.com/cosi/campaign/1935/vKlFQ43feFy9cC_VMpHFXVQkJ0rid7A9.png", width=800, height=800)

##### We know this is a huge challenge for every government indeed. However we do not expect to update infrastructure for electrics cars in a random way.
##### As well as gas stations, these places should be located strategically. As well of different bunch of suppliers ready to bite a piece of cake from this market.

In [4]:
Image(url= "https://i.guim.co.uk/img/media/7d5dfcdf7a70afe5273fa65590fdb99fdcda1a97/0_89_3000_1800/master/3000.jpg?width=1200&quality=85&auto=format&fit=max&s=d9a6fc098390afea29a96f4c59da2adc", width=800, height=800)

### Problem

##### Let say I am businessman interested to invest some of my money on this incoming business in the city of Puebla, Mexico. 
##### The question is...
####  Where  are the best places to build them ? 
##### Thinking a little bit by walking on the customer shoes, Why should I go to other side of the country, if there is no way to charge my car properly and risk my trip back home?
##### In order to make it a profitable business, not only a charge station should be builded, but a complete net around the city. 

#####  Therefore, we need to choose a strategically places to locate this charges spots. 
#####  Here we can approach the problem in different perspectives: 

* By income (How much does a neighborhood earns in average)
* By main streets
* By most frequently visited places
* By law 
* By the already installed charge spots. 
* Between others


##### On this analysis, we will choose the "By most frequently visited places" focus and use the Foursquare applications in order to find those places. 
##### The main hypothesis comes as:
####  It is more likely that people recharge their cars in more crowded places. Not only for business is better but also for accessibility of people.


### Data approach and methodology

##### Now some important considerations to take into account from venues: 

* We will focus on the city of Puebla. Taking into account a radio of 50 km around the main coordinate from Puebla
* Charge time for an average to go back home in the surrounding of Puebla city: At least 1 hr. 
* As shown before it doesn't make sense to consider places "Take n' go" if the customer will not spend enough time to charge it's car 
* We considered there is an electric infrastructure already on the selected boroughs.


#### Methodology: 

1. Find a link where to get Puebla City boroughs: 

https://codigospostales.nte.mx/poblados-de-Puebla-estado-de-puebla.html

2. Clean the data
   2.1 Load the set in the correct format for this information 
   2.2 Import coordinates
   2.3 Clean out unavailable data from geocoder
   2.4 Clean out data from coordinates out of range of 50 km around Puebla
   
3. Venues in Puebla 
   3.1 Load the venues in Puebla
   3.2 Take a look on the most frequently places. 
   3.2 Filter those venues which are not useful for the analysis.

4. K-means
   4.1 One hot data
   4.2 Get the top venues per area
   4.3 Get the K-means 
   4.4 Plot the venues per cluster
   
5. Analysis and Conclusions
   


In [5]:
Image(url= "https://www.honda.co.uk/engineroom/electric/ev/the-ultimate-electric-car-faq/assets/o2fBPuWzeW/batteriesh.gif", width=800, height=800)

## Development

In [1]:
#Required libraries for development
!pip install bs4
#!pip install requests

import numpy as np # library to handle data in a vectorized manner
import requests  # this module helps us to download a web page
import pandas as pd
import json # library to handle JSON files
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

!conda install -c conda-forge geocoder --yes  
import geocoder
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from bs4 import BeautifulSoup # this module helps in web scrapping.

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

#import geocoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from IPython import display
from IPython.display import Image

print("Libs ok")

     |████████████████████████████████| 122kB 13.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         336 KB

The following NEW packages will be INSTALLED:

  geocode

In [2]:
url0 = "https://codigospostales.nte.mx/poblados-de-Puebla-estado-de-puebla.html"
dataframe_list = pd.read_html(url0, flavor='bs4')
len(dataframe_list)

Poblacion_P = dataframe_list[0]
Poblacion_P.columns = ['Asentamiento','Nombre Asentamiento','Municipio','Estado','Codigo Postal']
Poblacion_P.head()

,Asentamiento,Nombre Asentamiento,Municipio,Estado,Codigo Postal
0,Colonia,15 de Septiembre,Puebla,Puebla,72227
1,Colonia,16 de Septiembre Norte,Puebla,Puebla,72230
2,Colonia,16 de Septiembre Sur,Puebla,Puebla,72474
3,Colonia,18 de Marzo,Puebla,Puebla,72595
4,Colonia,2 de Marzo,Puebla,Puebla,72227


In [3]:
#url1 = "https://codigospostales.nte.mx/poblados-de-San-Pedro-Cholula-estado-de-puebla.html"
#dataframe_list = pd.read_html(url1, flavor='bs4')
#len(dataframe_list)

#Poblacion_C = dataframe_list[0]
#Poblacion_C.columns = ['Asentamiento','Nombre Asentamiento','Municipio','Estado','Codigo Postal']
#Poblacion_C.head()

#frames = [Poblacion_P,Poblacion_C]
#result = pd.concat(frames)

ordenado = Poblacion_P

ordenado = ordenado.drop(['Asentamiento','Estado'], axis=1)
ordenado = ordenado.rename(columns = {'Nombre Asentamiento': 'Neighborhood', 'Municipio': 'Borough'}, inplace = False)
ordenado.head()

,Neighborhood,Borough,Codigo Postal
0,15 de Septiembre,Puebla,72227
1,16 de Septiembre Norte,Puebla,72230
2,16 de Septiembre Sur,Puebla,72474
3,18 de Marzo,Puebla,72595
4,2 de Marzo,Puebla,72227


In [5]:
puebla_coor = pd.DataFrame(columns = ['Latitude','Longitude'])

for row in ordenado['Neighborhood']:
        g = geocoder.osm(row + ", Puebla")
        v = g.latlng
        
        if v == None:
           v = [0,0]     #If geocoder coudn't find a location, then we switch None for a 0 Value, in order to continue with appending function
        
        puebla_coor = puebla_coor.append({'Latitude':v[0], 'Longitude':v[1]}, ignore_index=True)
        #print(v)

[19.0191096, -98.220416]
[24.0514963, -104.5929819]
[18.9947005, -98.2192318]
[18.968889, -98.160278]
[19.0519951, -98.2547964]
[23.04247685, -82.39901102935316]
[0, 0]
[0, 0]
[0, 0]
[20.1119433, -98.7049461]
[20.1119433, -98.7049461]
[19.111389, -98.147222]
[19.085461, -98.1409163]
[19.0411077, -98.187082]
[19.0175849, -98.2255591]
[25.28097425, -111.78558230731767]
[0, 0]
[-36.8007201, -59.8866144]
[19.0157056, -98.1267635]
[19.3168261, -99.2496896]
[25.4365665, -102.1653296]
[18.9759011, -98.2650688]
[20.2385082, -98.210237]
[19.0358442, -98.2329628]
[20.231389, -98.044444]
[19.156944, -98.453333]
[0, 0]
[20.133611, -97.201667]
[20.6289157, -87.0700683]
[22.006233, -99.0283866]
[20.495833, -97.577778]
[18.9619914, -98.2305341]
[0, 0]
[18.8870238, -96.9742081]
[0, 0]
[32.4095998, -117.0321033]
[19.0467239, -98.0606197]
[19.3281388, -98.9638175]
[0, 0]
[0, 0]
[19.0171709, -98.2523812]
[18.8776724, -98.4369957]
[19.1303348, -96.1761695]
[19.0587017, -98.2514552]
[0, 0]
[18.634722, -97.

In [4]:
#puebla_coor.to_csv("puebla_coor.csv")


#ordenado2 = ordenado.join(puebla_coor)
#ordenadoR = ordenado2
#ordenado2.to_csv("ordenado_CSV.csv")
#ordenadoR.head()

ordenado2 = pd.read_csv('ordenado_CSV.csv')
ordenado2.drop(columns=['Unnamed: 0'])
ordenado2.head()


,Unnamed: 0,Neighborhood,Borough,Codigo Postal,Latitude,Longitude
0,0,15 de Septiembre,Puebla,72227,19.019110,-98.220416
1,1,16 de Septiembre Norte,Puebla,72230,24.051496,-104.592982
2,2,16 de Septiembre Sur,Puebla,72474,18.994701,-98.219232
3,3,18 de Marzo,Puebla,72595,18.968889,-98.160278
4,4,2 de Marzo,Puebla,72227,19.051995,-98.254796


##### Now we have some irrelevant information which will not help to our analysis
##### First we will drop all those neighboords which Geocoder didn't recognize
##### Secondly, Geocoder might confuse some of the locations with another ones around the world.
##### Therefore, we will supreme all those places which are 1/2' Degree further than the Latitude and Logitude from Puebla (1' Degree ~ 111 km)

In [5]:
ordenado3 = ordenado2

LA_Puebla =  19.0413
LO_Puebla = -98.2062
Radio_P = 0.45   #Nearly 50 km round

ordenado3 = ordenado3[(ordenado3 != 0).all(1)]  # DropAll those neighboorhoods without location

ordenado3.drop(ordenado3[ordenado3['Latitude'] > (LA_Puebla + Radio_P)].index, inplace = True)
ordenado3.drop(ordenado3[ordenado3['Latitude'] < (LA_Puebla - Radio_P)].index, inplace = True)
ordenado3.drop(ordenado3[ordenado3['Longitude'] > (LO_Puebla + Radio_P)].index, inplace = True)
ordenado3.drop(ordenado3[ordenado3['Longitude'] < (LO_Puebla - Radio_P)].index, inplace = True)

ordenado3.drop_duplicates(subset=['Neighborhood'])
ordenado3 = ordenado3.reset_index(drop=True)
ordenado3.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e

,Unnamed: 0,Neighborhood,Borough,Codigo Postal,Latitude,Longitude
0,2,16 de Septiembre Sur,Puebla,72474,18.994701,-98.219232
1,3,18 de Marzo,Puebla,72595,18.968889,-98.160278
2,4,2 de Marzo,Puebla,72227,19.051995,-98.254796
3,11,6 de Junio,Puebla,72227,19.111389,-98.147222
4,12,8 de Diciembre,Puebla,72227,19.085461,-98.140916


In [6]:
map_Puebla = folium.Map(location = [LA_Puebla, LO_Puebla], zoom_start=10)  # create map of New York using latitude and longitude values

# add markers to map
for lat, lng, borough, neighborhood in zip(ordenado3['Latitude'], ordenado3['Longitude'], ordenado3['Borough'], ordenado3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Puebla)  
    
map_Puebla

In [7]:
#Foursquare crendentials

CLIENT_ID = 'C1WRPSVUXQ5MFGIRI333ZSGWFBLAKFVQPLWJYBKKCBLCXMV5' # your Foursquare ID
CLIENT_SECRET = '4LNXAPZACI4JYBSBY2AAHIWDCGN3LUTOEPWOH3TQ1BZZDYEH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C1WRPSVUXQ5MFGIRI333ZSGWFBLAKFVQPLWJYBKKCBLCXMV5
CLIENT_SECRET:4LNXAPZACI4JYBSBY2AAHIWDCGN3LUTOEPWOH3TQ1BZZDYEH


In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

#We will use the average location of the coordinates localiton
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    LA_Puebla,    #Latitude
    LO_Puebla,    #Longitude
    radius, 
    LIMIT)

results = requests.get(url2).json()

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


,name,categories,lat,lng
0,BRICO Pizzería Restaurant,Pizza Place,19.043550,-98.203515
1,Miel de Agave,Bar,19.042566,-98.200806
2,Super Paletería Mary Barragán,Ice Cream Shop,19.038942,-98.201833
3,Todo Rock,Tattoo Parlor,19.043225,-98.209372
4,El Mural de los Poblanos,Mexican Restaurant,19.042215,-98.199778
5,Café Colibri,Café,19.045592,-98.203850
6,Augurio,Mexican Restaurant,19.040761,-98.199114
7,Beso de Malta,Bar,19.040322,-98.199036
8,El Sueño Spa y Hotel,Hotel,19.041057,-98.199411
9,Profética - Casa de la Lectura,Coffee Shop,19.042934,-98.201720


In [10]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 41 uniques categories.


In [11]:
print (nearby_venues['categories'].value_counts())

Mexican Restaurant               21
Hotel                             7
Taco Place                        5
Coffee Shop                       5
Bar                               5
Café                              5
Candy Store                       4
Historic Site                     3
Steakhouse                        3
Ice Cream Shop                    3
Bakery                            3
Italian Restaurant                2
Plaza                             2
Park                              2
Church                            2
Pizza Place                       2
Department Store                  2
Gym / Fitness Center              1
Library                           1
Boutique                          1
French Restaurant                 1
Art Museum                        1
Sporting Goods Shop               1
Vegetarian / Vegan Restaurant     1
Chocolate Shop                    1
Event Space                       1
Sports Bar                        1
Cosmetics Shop              

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
Puebla_venues = getNearbyVenues(names=ordenado3['Neighborhood'],
                                   latitudes=ordenado3['Latitude'],
                                   longitudes=ordenado3['Longitude']
                                  )

16 de Septiembre Sur
18 de Marzo
2 de Marzo
6 de Junio
8 de Diciembre
Acocota
Adolfo López Mateos
Álamos Haras
Alcanfores
Alpha 2
Álvaro Obregón
Ampliación Balcones del sur
Ampliación Reforma
Angelopolis
Antigua Francisco Villa
Anzures
Arboledas de Loma Bella
Arboledas del Sur
Artículo Primero Constitucional
Balcones del Sur
Barranca Honda
Barrio de Santa Anita
Barrio San Juan (San Francisco Totimehuacan)
Barrio San Miguel
Barrios de Santa Catarina
Bellas Artes
Benito Juárez
Benito Juárez
Benito Juárez
Bosques de Amalucan
Bosques de Amalucan 1ra Sección
Bosques de Angelopolis
Bosques de Chapultepec
Bosques de la Cañada
Bosques de los Angeles
Bosques de Manzanilla
Bosques de Santa Anita
Britania
Buenavista Tetela
Cabañas del Lago
Calderón (Crucero el Oásis)
Camino Real
Carmen Huexotitla
Central de Abastos
Centro Comercial Puebla
Centro Cruz del Sur
Chula Vista
Cleotilde Torres
Club Britania
Club de Golf
Club de Golf las Fuentes
Club de Golf Puebla
Colibrí
Colorines
Concepción Sur
Consti

In [14]:
Puebla_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,16 de Septiembre Sur,18.994701,-98.219232,El Taco Hidalguense,18.995973,-98.218067,Taco Place
1,16 de Septiembre Sur,18.994701,-98.219232,Burrito Loco,18.996484,-98.220291,Burrito Place
2,16 de Septiembre Sur,18.994701,-98.219232,Oxxo,18.997288,-98.222876,Convenience Store
3,16 de Septiembre Sur,18.994701,-98.219232,El Tiburón,18.995349,-98.220820,Seafood Restaurant
4,16 de Septiembre Sur,18.994701,-98.219232,Comex,18.996424,-98.218716,Outdoor Supply Store


In [15]:
print('There are {} uniques categories.'.format(len(Puebla_venues['Venue Category'].unique())))

There are 261 uniques categories.


In [17]:
Top25 = Puebla_venues['Venue Category'].value_counts()
#Top25 = Top25[0:25] # Considering the top 25 most common places
#Top25 = Top25.value_counts().rename_axis('unique_values').reset_index(name='counts')
Top25

Mexican Restaurant                 400
Taco Place                         336
Convenience Store                  209
Coffee Shop                        124
Restaurant                         112
Seafood Restaurant                  93
Hotel                               92
Pharmacy                            86
Café                                85
Pizza Place                         81
Bar                                 71
Gym / Fitness Center                69
Italian Restaurant                  49
Gym                                 45
Burger Joint                        44
Bakery                              42
Steakhouse                          42
Ice Cream Shop                      42
Sandwich Place                      37
Park                                36
Shopping Mall                       36
Candy Store                         33
Sushi Restaurant                    30
Garden                              29
Food Truck                          28
Fried Chicken Joint      

In [30]:
#Now after some review, we will notice, that in order to charge a car electric, we will need to take out all those places which are "Take and go" and just those places
# where someone is likely to stay longer than an hour. And also those places where people use to drink, you know, safety first.
#Of course, for precisely information a deeper research on how much time does people expend on these venues might be helpful
PV = Puebla_venues


PV.drop(PV.index[PV['Venue Category'] == 'Taco Place'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Pizza Place'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Convenience Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Pharmacy'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bakery'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Ice Cream Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Candy Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Snack Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Food Truck'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bar'], inplace = True)         #We want to avoid driking while driving
PV.drop(PV.index[PV['Venue Category'] == 'Fried Chicken Joint'], inplace = True)   
PV.drop(PV.index[PV['Venue Category'] == 'Juice Bar'], inplace = True) 
PV.drop(PV.index[PV['Venue Category'] == 'Brewery'], inplace = True) 
PV.drop(PV.index[PV['Venue Category'] == 'Pet Store'], inplace = True) 
PV.drop(PV.index[PV['Venue Category'] == 'Liquor Store'], inplace = True) 
PV.drop(PV.index[PV['Venue Category'] == 'Beer Garden'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Paper / Office Supplies Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Hot Dog Joint'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Electronics Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Shoe Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Sports Bar'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Cosmetics Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Big Box Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Cocktail Bar'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Shipping Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Mobile Phone Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Arts & Crafts Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Flower Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Cupcake Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Donut Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Sporting Goods Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Irish Pub'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Pastry Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Athletics & Sports'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Beer Bar'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Dive Bar'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Speakeasy'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Food Stand'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bus Station'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Stables'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Frozen Yogurt Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bubble Tea Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Fountain'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Burrito Place'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Video Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Print Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Pub'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Video Game Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Toy / Game Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Butcher'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Smoke Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Gift Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Food & Drink Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bagel Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'ATM'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Business Service'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Wine Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Health Food Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Gourmet Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bus Stop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Volcano'], inplace = True)  #Definetly not there
PV.drop(PV.index[PV['Venue Category'] == 'Outdoor Supply Sotre'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == "Men's Store"], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Mountain'], inplace = True) #Neither here
PV.drop(PV.index[PV['Venue Category'] == 'Fish & Chips Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Record Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Creperie'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Hobby Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Tailor Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Monument / Landmark'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Cheese Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Jewelry Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Track'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Memorial Site'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == "Women's Store"], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Whisky Bar'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Public Art'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Heliport'], inplace = True)   #Cars can not fly yet
PV.drop(PV.index[PV['Venue Category'] == 'Herbs $ Spices Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Street Food Gathering'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Camera Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Fish Market'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Moving Target'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Baby Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Beer Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Lingerie Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Home Service'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Drugstore'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Organic Grocery'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Tiki Bar'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Supplement Shop'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Bridge'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Winery'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Outdoor Supply Store'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Boutique'], inplace = True)
PV.drop(PV.index[PV['Venue Category'] == 'Karaoke Bar'], inplace = True)


PV.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,16 de Septiembre Sur,18.994701,-98.219232,El Tiburón,18.995349,-98.220820,Seafood Restaurant
9,16 de Septiembre Sur,18.994701,-98.219232,Salon Jardin Angelis,18.992238,-98.217888,Garden Center
14,2 de Marzo,19.051995,-98.254796,Salón Zavaleta,19.055414,-98.255461,Lounge
15,2 de Marzo,19.051995,-98.254796,Star GYM Fitness,19.053075,-98.257794,Gym / Fitness Center
17,2 de Marzo,19.051995,-98.254796,Centro de Acondicionamiento Físico,19.054295,-98.257227,Gym


In [31]:
Puebla_onehot = pd.get_dummies(PV[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Puebla_onehot['Neighborhood'] = PV['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Puebla_onehot.columns[-1]] + list(Puebla_onehot.columns[:-1])
Puebla_onehot = Puebla_onehot[fixed_columns]

Puebla_onehot.head()

,Yucatecan Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baseball Field,Bath House,Bed & Breakfast,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Cable Car,Cafeteria,Café,Campground,Caribbean Restaurant,Casino,Chinese Restaurant,Church,Circus,City,City Hall,Clothing Store,Coffee Shop,College Administrative Building,College Auditorium,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Doctor's Office,Dog Run,Dry Cleaner,Empanada Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food Court,Football Stadium,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Go Kart Track,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Library,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Outdoor Event Space,Paintball Field,Park,Pie Shop,Planetarium,Playground,Plaza,Pool,Pool Hall,Post Office,Racetrack,Ramen Restaurant,Recreation Center,Rental Service,Resort,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sports Club,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swim School,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tourist Information Center,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Yoga Studio
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16 de Septiembre Sur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16 de Septiembre Sur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2 de Marzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2 de Marzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
Puebla_grouped = Puebla_onehot.groupby('Neighborhood').mean().reset_index()
Puebla_grouped.head()

,Neighborhood,Yucatecan Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Entertainment,Asian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Baseball Field,Bath House,Bed & Breakfast,Belgian Restaurant,Bike Trail,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Cable Car,Cafeteria,Café,Campground,Caribbean Restaurant,Casino,Chinese Restaurant,Church,Circus,City,City Hall,Clothing Store,Coffee Shop,College Administrative Building,College Auditorium,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Doctor's Office,Dog Run,Dry Cleaner,Empanada Restaurant,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food Court,Football Stadium,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Go Kart Track,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hot Spring,Hotel,Hotel Bar,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Library,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Outdoor Event Space,Paintball Field,Park,Pie Shop,Planetarium,Playground,Plaza,Pool,Pool Hall,Post Office,Racetrack,Ramen Restaurant,Recreation Center,Rental Service,Resort,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sports Club,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Swim School,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Restaurant,Tourist Information Center,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Warehouse Store,Water Park,Wine Bar,Wings Joint,Yoga Studio
0,16 de Septiembre Sur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2 de Marzo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.20,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acocota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.000000,0.0,

In [33]:
#Top 3 Common Venues per neigboorhood

num_top_venues = 5

for hood in Puebla_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Puebla_grouped[Puebla_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----16 de Septiembre Sur----
                venue  freq
0  Seafood Restaurant   0.5
1       Garden Center   0.5
2     Motorcycle Shop   0.0
3           Multiplex   0.0
4              Museum   0.0


----2 de Marzo----
                  venue  freq
0          Soccer Field   0.4
1  Gym / Fitness Center   0.2
2                   Gym   0.2
3                Lounge   0.2
4  Yucatecan Restaurant   0.0


----Acocota----
                venue  freq
0  Mexican Restaurant  0.40
1               Hotel  0.08
2                Park  0.04
3      Farmers Market  0.04
4              Church  0.04


----Adolfo López Mateos----
                 venue  freq
0   Mexican Restaurant  0.43
1                  Gym  0.07
2  Japanese Restaurant  0.07
3         Burger Joint  0.07
4   Rock Climbing Spot  0.07


----Alcanfores----
                  venue  freq
0    Mexican Restaurant   1.0
1  Yucatecan Restaurant   0.0
2       Paintball Field   0.0
3                Museum   0.0
4           Music Venue   0.0


----Alpha

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Puebla_grouped['Neighborhood']

for ind in np.arange(Puebla_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Puebla_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,16 de Septiembre Sur,Garden Center,Seafood Restaurant,Yoga Studio,Food,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
1,2 de Marzo,Soccer Field,Gym / Fitness Center,Gym,Lounge,Yoga Studio,Event Service,Farmers Market,Farm,Falafel Restaurant,Factory
2,Acocota,Mexican Restaurant,Hotel,Farmers Market,Italian Restaurant,City Hall,Garden,Snack Place,Brazilian Restaurant,Seafood Restaurant,Church
3,Adolfo López Mateos,Mexican Restaurant,Japanese Restaurant,Burger Joint,Seafood Restaurant,Baseball Field,Rock Climbing Spot,Health & Beauty Service,Gym,Gymnastics Gym,Diner
4,Alcanfores,Mexican Restaurant,Event Service,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space


In [44]:
kclusters = 7

Puebla_grouped_clustering = Puebla_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Puebla_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kint = kmeans.labels_
kint

array([1, 6, 2, 2, 0, 4, 4, 4, 4, 4, 4, 4, 4, 1, 6, 4, 0, 4, 4, 4, 2, 2,
       2, 2, 2, 3, 4, 4, 1, 4, 2, 1, 3, 4, 4, 2, 2, 2, 2, 4, 4, 4, 4, 2,
       2, 2, 4, 4, 4, 6, 2, 2, 2, 2, 2, 2, 4, 2, 2, 1, 2, 4, 4, 5, 1, 2,
       4, 4, 2, 4, 4, 4, 4, 4, 1, 2, 2, 4, 0, 2, 0, 0, 0, 4, 4, 2, 1, 2,
       1, 4, 4, 4, 6, 2, 4, 1, 2, 2, 2, 2, 2, 4, 4, 4, 5, 4, 1, 2, 4, 4,
       4, 2, 2, 4, 4, 4, 4, 4, 0, 0, 4, 4, 4, 2, 2, 4, 0, 4, 2, 4, 4, 6,
       4, 0, 6, 4, 2, 2, 2, 4, 4, 4, 4, 6, 0, 2, 2, 4, 2, 4, 2, 4, 2, 4,
       6, 2, 6, 6, 2, 4, 4, 4, 2, 6, 2, 4, 4, 2, 1, 2, 4, 4, 1, 2, 2, 4,
       4, 4, 1, 2, 5, 4, 6, 4, 2, 4, 2, 0, 4, 2, 2, 4, 1, 4, 0, 2, 2, 4,
       4, 2, 2, 4, 4, 2, 4, 0, 2, 4, 4, 5, 5, 2, 1, 4, 4, 2, 4, 4, 4],
      dtype=int32)

In [45]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kint)
#neighborhoods_venues_sorted['Cluster Labels'] = list(kint)
Puebla_merged = ordenado3

#merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Puebla_merged =  Puebla_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Puebla_merged['Cluster Labels'] = Puebla_merged['Cluster Labels'].fillna(0)
Puebla_merged['Cluster Labels'] = Puebla_merged['Cluster Labels'].apply(np.int64)

Puebla_merged.dropna(inplace=True)

Puebla_merged.head()

,Unnamed: 0,Neighborhood,Borough,Codigo Postal,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,16 de Septiembre Sur,Puebla,72474,18.994701,-98.219232,1,Garden Center,Seafood Restaurant,Yoga Studio,Food,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
2,4,2 de Marzo,Puebla,72227,19.051995,-98.254796,6,Soccer Field,Gym / Fitness Center,Gym,Lounge,Yoga Studio,Event Service,Farmers Market,Farm,Falafel Restaurant,Factory
5,13,Acocota,Puebla,72377,19.041108,-98.187082,2,Mexican Restaurant,Hotel,Farmers Market,Italian Restaurant,City Hall,Garden,Snack Place,Brazilian Restaurant,Seafood Restaurant,Church
6,14,Adolfo López Mateos,Puebla,72240,19.017585,-98.225559,2,Mexican Restaurant,Japanese Restaurant,Burger Joint,Seafood Restaurant,Baseball Field,Rock Climbing Spot,Health & Beauty Service,Gym,Gymnastics Gym,Diner
8,21,Alcanfores,Puebla,72583,18.975901,-98.265069,0,Mexican Restaurant,Event Service,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space


In [46]:
map_clusters = folium.Map(location=[LA_Puebla, LO_Puebla], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
    
#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Puebla_merged ['Latitude'], Puebla_merged ['Longitude'], Puebla_merged ['Neighborhood'], Puebla_merged ['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analyzing Clusters

In [57]:
#Cluster 1 - Red ones
C1 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 0, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C1.head()

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Alcanfores,-98.265069,0,Mexican Restaurant,Event Service,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space
25,Bellas Artes,-98.304688,0,Mexican Restaurant,Pool Hall,Plaza,Yoga Studio,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory
97,Héroe de Nacozari,-98.200882,0,Mexican Restaurant,Museum,Park,Yoga Studio,Event Space,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory
104,Ignacio Mariscal,-98.201960,0,Mexican Restaurant,Event Service,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space
106,INFONAVIT Fuentes de San Bartolo,-98.234868,0,Mexican Restaurant,Café,BBQ Joint,Food Court,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory


In [58]:
#Cluster 2 - Purple
C2 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 1, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C2.head()

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,16 de Septiembre Sur,-98.219232,1,Garden Center,Seafood Restaurant,Yoga Studio,Food,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
22,Barrio San Juan (San Francisco Totimehuacan),-98.203869,1,Steakhouse,Café,Garden,Seafood Restaurant,Yoga Studio,Event Space,Farmers Market,Farm,Falafel Restaurant,Factory
42,Carmen Huexotitla,-98.262747,1,Seafood Restaurant,Breakfast Spot,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
45,Centro Cruz del Sur,-98.263576,1,Breakfast Spot,Seafood Restaurant,Food,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
80,Everest,-98.112986,1,Gym / Fitness Center,Seafood Restaurant,Bath House,Yoga Studio,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory


In [59]:
#Cluster 3 - Blue Navy
C3 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 2, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C3.head()

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Acocota,-98.187082,2,Mexican Restaurant,Hotel,Farmers Market,Italian Restaurant,City Hall,Garden,Snack Place,Brazilian Restaurant,Seafood Restaurant,Church
6,Adolfo López Mateos,-98.225559,2,Mexican Restaurant,Japanese Restaurant,Burger Joint,Seafood Restaurant,Baseball Field,Rock Climbing Spot,Health & Beauty Service,Gym,Gymnastics Gym,Diner
32,Bosques de Chapultepec,-98.209527,2,Mexican Restaurant,Restaurant,Coffee Shop,Hotel,Vegetarian / Vegan Restaurant,Burger Joint,Seafood Restaurant,Steakhouse,Sandwich Place,Museum
33,Bosques de la Cañada,-98.136329,2,Restaurant,Mexican Restaurant,Supermarket,Market,Seafood Restaurant,Yoga Studio,Event Space,Farmers Market,Farm,Falafel Restaurant
34,Bosques de los Angeles,-98.136329,2,Restaurant,Mexican Restaurant,Supermarket,Market,Seafood Restaurant,Yoga Studio,Event Space,Farmers Market,Farm,Falafel Restaurant


In [52]:
#Cluster 4 - Green Blue 
C4 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 3, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C4

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Buenavista Tetela,-98.174375,3,Campground,Yoga Studio,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space
46,Chula Vista,-98.174037,3,Campground,Yoga Studio,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space


In [60]:
#Cluster 5 - Cyan
C5 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 4, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C5.head()

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Alpha 2,-98.232963,4,Coffee Shop,Nightclub,Shopping Mall,Restaurant,Seafood Restaurant,Mexican Restaurant,Steakhouse,Furniture / Home Store,Department Store,Clothing Store
11,Ampliación Balcones del sur,-98.230534,4,Garden Center,Lounge,Auto Garage,Yoga Studio,Event Space,Flea Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
12,Ampliación Reforma,-98.060620,4,Restaurant,Event Space,Yoga Studio,Empanada Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
13,Angelopolis,-98.252381,4,Shopping Mall,Hotel,Italian Restaurant,Coffee Shop,Mexican Restaurant,Sandwich Place,Spa,Spanish Restaurant,Steakhouse,Restaurant
14,Antigua Francisco Villa,-98.436996,4,Hot Spring,Restaurant,Lounge,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit


In [62]:
#Cluster 6 - Yellow Mustard
C6 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 5, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C6.head()

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
84,Geovillas del Sur,-98.256769,5,Restaurant,Park,Yoga Studio,Empanada Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space
85,Geovillas del Sur,-98.256769,5,Restaurant,Park,Yoga Studio,Empanada Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space
133,La Noria,-98.220755,5,Gym,Park,Yoga Studio,Empanada Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
254,Santa Catarina,-98.161389,5,Park,Yoga Studio,Empanada Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space
255,Santa Catarina,-98.161389,5,Park,Yoga Studio,Empanada Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit,Event Space


In [61]:
#Cluster 7 - Orange
C7 = Puebla_merged.loc[Puebla_merged['Cluster Labels'] == 6, Puebla_merged.columns[[1] + list(range(5, Puebla_merged.shape[1]))]]
C7.head()

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,2 de Marzo,-98.254796,6,Soccer Field,Gym / Fitness Center,Gym,Lounge,Yoga Studio,Event Service,Farmers Market,Farm,Falafel Restaurant,Factory
21,Barrio de Santa Anita,-98.187961,6,Department Store,Soccer Field,Yoga Studio,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
69,El Mayorazgo,-98.239884,6,Soccer Field,Mexican Restaurant,Nightclub,Gymnastics Gym,Golf Course,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
117,Joaquín Colombres,-98.258875,6,Soccer Field,Auto Garage,Yoga Studio,Event Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
174,Mayorazgo,-98.239884,6,Soccer Field,Mexican Restaurant,Nightclub,Gymnastics Gym,Golf Course,Farmers Market,Farm,Falafel Restaurant,Factory,Exhibit
